In [2]:
import requests
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
from bs4 import UnicodeDammit
import numpy as np
import pandas as pd
from nltk.tokenize import sent_tokenize
import language_check
import re

In [3]:
def get_sentences(url, find, headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36'}):
    r = requests.get(url, headers=headers)
    # make sure that the page exist
    if r.status_code == 200:
        html = r.content
        try:
            dammit = UnicodeDammit(html)
            #print(dammit.unicode_markup)
            encoding = dammit.original_encoding
            soup = BeautifulSoup(html, 'lxml', exclude_encodings=[encoding])
            new_encoding = soup.original_encoding
            #print(soup.original_encoding)
            soup = BeautifulSoup(html, 'lxml',  exclude_encodings=[new_encoding])
            #print(soup.original_encoding)
            title = soup.find('h1')
            if title is not None:
                title_text = title.text.strip()
            #article = soup.find(find)   
            article = soup.find_all(find)
            new_article = []
            for i in article:
                dammit = UnicodeDammit(i.text.strip().encode(), smart_quotes_to="ascii").unicode_markup
                new_article.append((dammit.encode('ascii', 'ignore')).decode("utf-8"))
            sentences = []
            for i in new_article:
                sentences.append(sent_tokenize(i))
            sentences = sum(sentences, [])
            return sentences
        except TypeError:
            print("Type error with url: ", url)
    
def multi_sites(url_list, find):
    all_sents = []
    new_url = []
    for i in url_list:
        this_url_sents = get_sentences(i, find)
        if not isinstance(this_url_sents, type(None)):
            new_url.append(i)
            all_sents.append(this_url_sents)
    all_sents = sum(all_sents,[])
    return all_sents, new_url

In [4]:
def get_sentences_container(url, find, headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36'}):
    r = requests.get(url, headers=headers)
    # make sure that the page exist
    if r.status_code == 200:
        html = r.content
        try:
            dammit = UnicodeDammit(html)
            #print(dammit.unicode_markup)
            encoding = dammit.original_encoding
            soup = BeautifulSoup(html, 'lxml', exclude_encodings=[encoding])
            new_encoding = soup.original_encoding
            #print(soup.original_encoding)
            soup = BeautifulSoup(html, 'lxml',  exclude_encodings=[new_encoding])
            #print(soup.original_encoding)
            title = soup.find('h1')
            if title is not None:
                title_text = title.text.strip()
            #article = soup.find(find)   
            article = soup.find_all(find).text
            new_article = []
            for i in article:
                dammit = UnicodeDammit(i.text.strip().encode(), smart_quotes_to="ascii").unicode_markup
                new_article.append((dammit.encode('ascii', 'ignore')).decode("utf-8"))
            sentences = []
            for i in new_article:
                sentences.append(sent_tokenize(i))
            sentences = sum(sentences, [])
            return sentences
        except TypeError:
            print("Type error with url: ", url)
    
def multi_sites_container(url_list, find):
    all_sents = []
    new_url = []
    for i in url_list:
        this_url_sents = get_sentences_container(i, find)
        if not isinstance(this_url_sents, type(None)):
            new_url.append(i)
            all_sents.append(this_url_sents)
    all_sents = sum(all_sents,[])
    return all_sents, new_url

In [5]:

def save_csv(batch, file_name):
    batch = np.asarray(batch)
    pd_df = pd.DataFrame(batch)
    pd_df.to_csv(file_name, index = False, header = False)

In [6]:
def grammar_check(batch):
    grammar_tool = language_check.LanguageTool('en-GB')
    checked = []
    for i in batch:
        corrected = ''
        matches = grammar_tool.check(i)
        checked.append(language_check.correct(i, matches))
    return checked


In [6]:
first_batch_url =['http://www.dating-relationship-advice-for-women.com/what-do-men-want/', 
                  'http://www.dating-relationship-advice-for-women.com/does-he-like-me/',
                  'http://www.dating-relationship-advice-for-women.com/seducing-men/',
                  'http://www.dating-relationship-advice-for-women.com/signs-of-flirting/',
                  'http://www.dating-relationship-advice-for-women.com/attract-a-guy.html',
                  'http://www.dating-relationship-advice-for-women.com/is-he-the-one/',
                  'http://www.dating-relationship-advice-for-women.com/money-or-love/',
                  'http://www.dating-relationship-advice-for-women.com/sugar-daddy.html',
                  'http://www.dating-relationship-advice-for-women.com/shy-guy/',
                  'http://www.dating-relationship-advice-for-women.com/too-old-to-date/'
                  ]

In [7]:
first_batch, first_url_list = multi_sites(first_batch_url, "p")

In [85]:
len(first_batch)

653

In [97]:
first_batch

['What do men want is a brilliant question that has confused women since the beginning of time.',
 'Men are strange, yet often simple creatures who are on a constant quest to define themselves in a world that no longer cares what they accomplish.',
 'There are no valiant causes to fight for.',
 'There are no more glaring injustices to stand against.',
 'Even if he went off to fight in any of the current worldwide conflicts, he would not really be looked upon with pride.',
 "He's more likely to be met with the sadness and grief that war creates.",
 'Men are lost.',
 'The things that used to mean something have lost their meaning so meaningless pursuits have taken their place.',
 'All you have to do is listen to the messages of pop culture to see that disrespect and criminal behavior are the new pursuits.',
 'If you asked a man today what he wants, he might not even be able to tell you.',
 'This leads us to you, the last true and traditional pursuit that a man can be proud of.',
 'The an

In [98]:
first_url_list

['http://www.dating-relationship-advice-for-women.com/what-do-men-want/',
 'http://www.dating-relationship-advice-for-women.com/does-he-like-me/',
 'http://www.dating-relationship-advice-for-women.com/seducing-men/',
 'http://www.dating-relationship-advice-for-women.com/signs-of-flirting/',
 'http://www.dating-relationship-advice-for-women.com/is-he-the-one/',
 'http://www.dating-relationship-advice-for-women.com/money-or-love/',
 'http://www.dating-relationship-advice-for-women.com/shy-guy/',
 'http://www.dating-relationship-advice-for-women.com/too-old-to-date/']

In [46]:
now_what = grammar_check(first_batch)

In [47]:
now_what

['What do men want is a brilliant question that has confused women since the beginning of time.',
 'Men are strange, yet often simple creatures who are on a constant quest to define themselves in a world that no longer cares what they accomplish.',
 'There are no valiant causes to fight for.',
 'There are no more glaring injustices to stand against.',
 'Even if he went off to fight in any of the current worldwide conflicts, he would not really be looked upon with pride.',
 "He's more likely to be met with the sadness and grief that war creates.",
 'Men are lost.',
 'The things that used to mean something have lost their meaning so meaningless pursuits have taken their place.',
 'All you have to do is listen to the messages of pop culture to see that disrespect and criminal behaviour are the new pursuits.',
 'If you asked a man today what he wants, he might not even be able to tell you.',
 'This leads us to you, the last true and traditional pursuit that a man can be proud of.',
 'The a

In [39]:
save_csv(now_what, './first_batch.csv')

In [6]:
second_batch_url = ['http://www.dating-relationship-advice-for-women.com/how-to-be-attractive/',
                    'http://www.dating-relationship-advice-for-women.com/get-a-guy-to-ask-you-out/',
                    'http://www.dating-relationship-advice-for-women.com/first-date-tips/',
                    'http://www.dating-relationship-advice-for-women.com/signs-he-is-into-you/',
                    'http://www.dating-relationship-advice-for-women.com/dealing-with-jealousy/',
                    'http://www.dating-relationship-advice-for-women.com/how-to-spot-a-cheater/',
                    'http://www.dating-relationship-advice-for-women.com/signs-of-infidelity/',
                    'http://www.dating-relationship-advice-for-women.com/emotional-affair/',
                    'http://www.dating-relationship-advice-for-women.com/is-my-man-cheating/'
                   ]

In [7]:
second_batch, second_url_list = multi_sites(second_batch_url, "p")

In [15]:
len(second_batch)

728

In [16]:
second_batch

['When you look at it, the people who know how to be attractive get everything theyve always wanted in life.',
 'Were not just talking about being born the beautiful daughter of two famous supermodel parents.',
 'Were talking about true beauty.',
 'The kind of irresistible charisma of a person who has worked hard on making their lives incredible.',
 'Do you know people like this?',
 'Maybe its one of your friends.',
 'She walks into the room and steals everyones breath by the sheer force of her presence.',
 'Shes funny, smart, kind, and personable.',
 'You miss her when shes not there.',
 'She has all the great qualities that know just how to attract a man.',
 'Whats keeping you from being that person?',
 'She wasnt born charming!',
 'She didnt pop out of her mother and immediately start in with an anecdote about the cute guy who helped her find her binky.',
 'Being charismatic is something you have to learn.',
 'Although your genes make you crave fascinating people, they donEUR(TM)t g

In [18]:
second_url_list

['http://www.dating-relationship-advice-for-women.com/how-to-be-attractive/',
 'http://www.dating-relationship-advice-for-women.com/get-a-guy-to-ask-you-out/',
 'http://www.dating-relationship-advice-for-women.com/first-date-tips/',
 'http://www.dating-relationship-advice-for-women.com/signs-he-is-into-you/',
 'http://www.dating-relationship-advice-for-women.com/dealing-with-jealousy/',
 'http://www.dating-relationship-advice-for-women.com/how-to-spot-a-cheater/',
 'http://www.dating-relationship-advice-for-women.com/signs-of-infidelity/',
 'http://www.dating-relationship-advice-for-women.com/emotional-affair/',
 'http://www.dating-relationship-advice-for-women.com/is-my-man-cheating/']

In [49]:
now_what_second = grammar_check(second_batch)

In [50]:
save_csv(now_what_second, './second_batch.csv')

# New source site

In [32]:
third_batch_url = ['https://www.zoosk.com/date-mix/relationship-advice/relationship-advice-for-men-2/',
                   'https://www.zoosk.com/date-mix/dating-advice/body-language-of-men/',
                   'https://www.zoosk.com/date-mix/dating-advice/how-to-be-a-gentleman/',
                   'https://www.zoosk.com/date-mix/dating-advice/body-language-of-women/',
                   'https://www.zoosk.com/date-mix/over-50-dating/older-women-dating/',
                   'https://www.zoosk.com/date-mix/relationship-advice/breaking-up/should-i-break-up-with-my-girlfriend/',
                   'https://www.zoosk.com/date-mix/relationship-advice/why-women-pull-away/',
                   'https://www.zoosk.com/date-mix/dating-advice/single-women/',
                   'https://www.zoosk.com/date-mix/dating-advice/single-men/',
                   'https://www.zoosk.com/date-mix/single-life/flirting/is-he-flirting-with-me/',
                   'https://www.zoosk.com/date-mix/over-50-dating/single-men-over-50/',
                   'https://www.zoosk.com/date-mix/over-50-dating/single-women-over-50/',
                   'https://www.zoosk.com/date-mix/dating-advice/short-guys/',
                   'https://www.zoosk.com/date-mix/relationship-advice/older-man-younger-woman-relationship/',
                   'https://www.zoosk.com/date-mix/dating-advice/dating-advice-women/best-dating-advice-women/',
                   'https://www.zoosk.com/date-mix/over-50-dating/mature-women-dating/',
                   'https://www.zoosk.com/date-mix/dating-advice/dating-advice-men/best-dating-advice-men/',
                   'https://www.zoosk.com/date-mix/dating-advice/science-younger-women-date-older-men/',
                   'https://www.zoosk.com/date-mix/dating-advice/how-to-spot-a-player/']
                  

In [7]:
def div_sites(url, find):
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    sentences = []
    
    try:
        web_byte = urlopen(req).read()
        webpage = web_byte.decode('utf-8')
        dammit = UnicodeDammit(webpage)
        encoding = dammit.original_encoding
        soup = BeautifulSoup(webpage, 'lxml', exclude_encodings=[encoding])
        new_encoding = soup.original_encoding
        soup = BeautifulSoup(webpage, 'lxml',  exclude_encodings=[new_encoding])
        title = soup.find('h1')
        this_thing = ''
        if title is not None:
            title_text = title.text.strip()
            article = soup.find(find)  
            this_thing = article.find_all("p")
        new_sents = []
        for i in range(0, len(this_thing)):
            dammit = UnicodeDammit(this_thing[i].text.encode()).unicode_markup
            if isinstance(dammit, str):
                what = (dammit.encode('ascii', 'ignore')).decode("utf-8")
                new_sents.append(what)
        new_sents = grammar_check(new_sents)
        for i in new_sents:
            sentences.append(sent_tokenize(i))
        sentences = sum(sentences, [])
        return sentences
    except TypeError:
        print("Type error with url: ", url)
        
        
def div_multi_sites_container(url_list, find):
    all_sents = []
    new_url = []
    for i in url_list:
        this_url_sents = div_sites(i, find)
        if not isinstance(this_url_sents, type(None)):
            new_url.append(i)
            all_sents.append(this_url_sents)
    all_sents = sum(all_sents,[])
    return all_sents, new_url

In [33]:
testing, test_url = div_multi_sites_container(third_batch_url, ("div",{'class':'small-12 medium-7 columns'}))

In [40]:
len(testing)

1121

In [13]:
testing[:10]

['We all know that men and women handle various situations differently.',
 'So when it comes to dating advice, what might work for women, might not work so great for the guys.',
 'Were different, and those differences are what make our relationships fun and interesting and also kind of confusing sometimes.',
 'To help clear up the confusion, we asked different experts what they thought the best relationship advice for men waste things guys really need to know to understand women more and get the most of their relationships with them.',
 'Here the best relationship advice for men, according to experts across the field:',
 '1.',
 "Don't discredit her feelings.",
 'A lot of men have a tendency to invalidate their partners feelings.',
 "For example, they might say, That's a silly thing to be mad about, or I can't believe that upset you, explains premarital counsellor, Rake Lawson.",
 'Instead, try to validate your partners emotions by apologizing for being the source of that emotion withou

In [14]:
save_csv(testing, './third_batch.csv')

In [ ]:
fourth_batch_url = ['https://www.psychologytoday.com/us/blog/growing-friendships/201108/can-boys-and-girls-be-friends'


In [51]:
fifth_batch_url = [ 'https://www.psychologytoday.com/us/articles/200203/we-were-happier-moms-and-working-daughters',
                    'https://www.psychologytoday.com/us/blog/tech-support/201406/is-it-always-turf-war-adult-daughters-and-their-mothers',
                    'https://www.psychologytoday.com/us/blog/the-friendship-doctor/201005/can-mother-be-daughters-best-friend',
                    'https://www.psychologytoday.com/us/blog/singletons/201401/inside-every-mother-is-daughter',
                    'https://www.psychologytoday.com/us/blog/eat-write-love/201707/mother-daughter',
                    'https://www.psychologytoday.com/us/blog/the-legacy-distorted-love/201310/mothers-who-are-jealous-their-daughters',
                    'https://www.psychologytoday.com/us/articles/200105/the-mother-daughter-bond',
                    'https://www.psychologytoday.com/us/articles/200309/rules-dating-my-daughter']

In [17]:
fourth, fourth_url = div_multi_sites_container(fourth_batch_url, ("body",{'class':"html not-front not-logged-in one-sidebar sidebar-first page-node page-node- page-node-71736 node-type-blog-entry i18n-en-US user-role-anonymous-user chrome chrome70 mac desktop"}))

In [47]:
a = fourth[:26] +fourth[69:]

In [48]:
len(a)

71

In [37]:
fourth.index('Did you when you were growing up?')

26

In [52]:
fifth, fifth_url = div_multi_sites_container(fifth_batch_url, ("body",{'class':"html not-front not-logged-in one-sidebar sidebar-first page-node page-node- page-node-71736 node-type-blog-entry i18n-en-US user-role-anonymous-user chrome chrome70 mac desktop"}))

In [56]:
fifth

['The study, “My Daughter Has a Career; I Just Raised Babies,” used data gathered when the women were ages 18, 36 and 53, plus interviews with 16 mothers at age 59.',
 'Carr found that they developed “self-protective” strategies to rationalize the generation gap.',
 "The mothers attributed their daughters' success to education and career planning as well as talent and motivation.",
 'They also emphasized the downside: In pursuing careers, their daughters had created lives that were more stressful and unhappy than their own.',
 "But Carr's study, presented to the Gerontological Society of America, revealed some contradictions.",
 'Almost half the women were encouraged to attend college when they graduated from high school in 1957, and they had an average of 13 years of education, only one year less than their daughters.',
 'Seventy percent were employed at age 35.',
 '“They were telling stories as if they were the quintessential June Cleaver,” explains Carr.',
 "“But they weren't.” She 

In [57]:
sixth_batch_url = ['https://www.psychologytoday.com/us/blog/it-s-your-choice/201810/sons-in-search-fathers-fathers-in-search-sons',
                   'https://www.psychologytoday.com/us/blog/fixing-families/201103/fathers-sons-how-be-great-dad',
                   'https://www.psychologytoday.com/us/blog/hope-relationships/201404/9-tips-communicating-your-teenage-son',
                   'https://www.psychologytoday.com/us/blog/surviving-your-childs-adolescence/201011/the-challenge-fathering-adolescent-son',
                   'https://www.psychologytoday.com/us/articles/200505/the-myth-the-clueless-husband',
                   'https://www.psychologytoday.com/us/articles/199501/mamas-boys',
                   'https://www.psychologytoday.com/us/blog/the-heart-it-all/201101/shall-i-live-longer-if-i-quarrel-my-wife-or-if-i-have-no-wife-quarrel',
                   'https://www.psychologytoday.com/us/blog/let-their-words-do-the-talking/201312/buy-your-wife-vacuum-cleaner-christmas',
                   'https://www.psychologytoday.com/us/blog/it-s-man-s-and-woman-s-world/201310/beauty-status-and-the-trophy-wife-myth',
                   'https://www.psychologytoday.com/us/blog/the-puzzle-love/201506/why-wife-bonus-won-t-buy-you-security'
                  ]

In [58]:
sixth, sixth_url = div_multi_sites_container(sixth_batch_url, ("body",{'class':"html not-front not-logged-in one-sidebar sidebar-first page-node page-node- page-node-71736 node-type-blog-entry i18n-en-US user-role-anonymous-user chrome chrome70 mac desktop"}))

In [59]:
out = a + fifth + sixth

In [60]:
save_csv(out, './fourth_batch.csv')